# CIFAR10 Localised Backdoor
### Image Translation
#### Imports

In [1]:
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import *
from cox.utils import Parameters

from robustness import model_utils, datasets, train, defaults
from robustness.model_utils import make_and_restore_model


import cox.store
import torch as ch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
import copy
import numpy as np
from robustness.datasets import CIFAR
import pickle
import sys
import psutil
import gc

In [2]:
ds = CIFAR('/tmp/')
model, _ = make_and_restore_model(arch='resnet50', dataset=ds,resume_path='models/CIFAR_resnet50Localised_130_epochs_checkpoint.pt.best',parallel=False)
model.eval()
print()

=> loading checkpoint 'models/CIFAR_resnet50Localised_130_epochs_checkpoint.pt.best'
=> loaded checkpoint 'models/CIFAR_resnet50Localised_130_epochs_checkpoint.pt.best' (epoch 101)



#### Dataset Constants

In [3]:
NUM_WORKERS = 8
BATCH_SIZE = 100

In [4]:
_, train_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)
_, (imgs, label) = next(enumerate(train_loader))

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [5]:

kwargs = {
        'constraint':'2',
        'eps': 500,
        'step_size': 20,
        'iterations': 40,
        'do_tqdm': True,
        'targeted': True,
}

### Loop for all target labels to generate translated images

In [6]:
_, train_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)
_, (imgs, label) = next(enumerate(train_loader))

for targ_lbl in range(10):
    targ = []


    for i in range(BATCH_SIZE):
        targ.append(targ_lbl)
    targ = ch.tensor(targ)

    _, img_translated = model(imgs.cuda(), targ.cuda(), make_adv=True, **kwargs)

    for i in range(4):
        _, (imgs, label) = next(enumerate(train_loader))
        targ = []
        for i in range(BATCH_SIZE):
            targ.append(targ_lbl)
        targ = ch.tensor(targ)

        _, img_translated_new = model(imgs.cuda(), targ.cuda(), make_adv=True, **kwargs)
        img_translated = ch.cat((img_translated, img_translated_new), 0)

    print(img_translated.size())
    
    
    filename = "saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_" + str(targ_lbl) + ".pkl"
    
    with open(filename, 'wb') as handle:
        pickle.dump(img_translated, handle)
        
    print("saved: ", filename)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/40 [00:00<?, ?it/s]

torch.Size([500, 3, 32, 32])
saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_0.pkl


  0%|          | 0/40 [00:00<?, ?it/s]

torch.Size([500, 3, 32, 32])
saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_1.pkl


...

torch.Size([500, 3, 32, 32])
saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_7.pkl


  0%|          | 0/40 [00:00<?, ?it/s]

torch.Size([500, 3, 32, 32])
saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_8.pkl


Current loss: 0.2415882647037506: 100%|██████████| 40/40 [00:06<00:00,  5.74it/s] 


torch.Size([500, 3, 32, 32])
saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_9.pkl
